## Este código é somente uma ferramenta de validação, para garantir que não foram coletados acórdãos duplicados nem que há arquivos faltantes ou em excesso. 

In [1]:
import os
import pandas as pd
from typing import Set
import polars as pl
import os

In [4]:
def get_pdf_names(directory: str) -> Set[str]:
    """Retorna nomes dos arquivos PDF sem extensão."""
    return {os.path.splitext(f)[0] for f in os.listdir(directory) if f.lower().endswith('.pdf')}

def get_csv_ids(csv_path: str) -> Set[str]:
    """Retorna IDs únicos do CSV como strings."""
    return set(pd.read_csv(csv_path)['id'].dropna().astype(str))

def validate_files():
    """Valida se todos os IDs do CSV têm arquivos PDF correspondentes."""
    pdf_directory = "STF FINAL"
    csv_file = "jurisprudencia_data.csv"
    
    pdf_files = get_pdf_names(pdf_directory)
    csv_ids = get_csv_ids(csv_file)
    missing_files = csv_ids - pdf_files
    extra_files = pdf_files - csv_ids
    
    print(f"Total de IDs no CSV: {len(csv_ids)}")
    print(f"Total de PDFs na pasta: {len(pdf_files)}")
    
    if missing_files:
        print(f"\nArquivos faltantes ({len(missing_files)}):")
        for file_id in sorted(missing_files):
            print(f"  {file_id}")
    else:
        print("Todos os documentos do CSV estão presentes.")
    
    if extra_files:
        print(f"\nArquivos extras na pasta ({len(extra_files)}):")
        for file_id in sorted(extra_files):
            print(f"  {file_id}")
    else:
        print("Nenhum arquivo extra na pasta.")

def check_duplicates():
    """Verifica se existem valores duplicados na segunda coluna do arquivo parquet."""
    parquet_file = "documentos.parquet"
    
    if not os.path.exists(parquet_file):
        print(f"Arquivo não encontrado: {parquet_file}")
        return
    
    try:
        df = pl.read_parquet(parquet_file)
        second_column_name = df.columns[1]
        
        total_values = df.height
        unique_values = df[second_column_name].n_unique()
        duplicated_values = total_values - unique_values
        
        print(f"Coluna analisada: {second_column_name}")
        print(f"Total de valores: {total_values}")
        print(f"Valores únicos: {unique_values}")
        
        if duplicated_values > 0:
            print(f"Valores duplicados encontrados: {duplicated_values}")
            
            # Encontra linhas com valores duplicados na segunda coluna
            duplicated_df = df.filter(
                pl.col(second_column_name).is_in(
                    df.group_by(second_column_name)
                    .count()
                    .filter(pl.col("count") > 1)[second_column_name]
                )
            ).sort(second_column_name)
            
            print(f"\nInício dos textos duplicados:")
            for row in duplicated_df.iter_rows(named=True):
                text_preview = str(row[second_column_name])[:100] + "..." if len(str(row[second_column_name])) > 100 else str(row[second_column_name])
                print(f"ID: {row[df.columns[0]]} | Texto: {text_preview}")
        else:
            print("Nenhum valor duplicado encontrado na segunda coluna.")
            
    except Exception as e:
        print(f"Erro ao ler arquivo parquet: {str(e)}")




In [6]:
validate_files()

Total de IDs no CSV: 1108
Total de PDFs na pasta: 1108
Todos os documentos do CSV estão presentes.
Nenhum arquivo extra na pasta.


In [ ]:
check_duplicates()

Coluna analisada: texto_completo
Total de valores: 1084
Valores únicos: 1084
Nenhum valor duplicado encontrado na segunda coluna.
